In [49]:
from deeppavlov.core.models.component import Component
from deeppavlov.core.agent.agent import Agent
from deeppavlov.models.classifiers.vocab_intersection.vocab_intersection import VocabIntersection

In [4]:
class OneLiner(Component):
    def __init__(self, line, *args, **kwargs):
        self.line = line
    
    def __call__(self, batch, *args, **kwargs):
        return [(self.line, 1, None) for _ in batch]

In [5]:
hello = OneLiner('Hello')
world = OneLiner('world')
other = OneLiner("I don't understand, sorry")

In [63]:
class CustomSelector(Component):
    def __init__(self, *args, **kwargs):
        self.f = [
            VocabIntersection(["hi", "hello", "good day"]),
            VocabIntersection(["bye", "chao", "see you"]),
            lambda batch: [1. for _ in batch]
        ]
    
    def __call__(self, utterances, batch_history, *responses):
        tokens = [u.lower().split() for u in utterances]
        confidences = [f(tokens) for f in self.f]
        indexes = [c.index(max(c)) for c in zip(*confidences)]
        return [responses[i][0] for i, *responses in zip(indexes, *responses)]

In [64]:
agent = Agent([hello, world, other], skills_selector=CustomSelector())

In [65]:
agent(['Hello', 'Bye', 'Or not'])

['Hello', 'world', "I don't understand, sorry"]